# Machine Learning Experiment \#1

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

# Plotly imports
import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

from operator import add

import bisect
import datetime
import math
import random
import time

# Keras imports
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
#from keras.layers.core import Dense, Dropout
from keras.layers import Dense, Dropout, LSTM, Embedding

In [ ]:
from numpy.random import choice

In [ ]:
N = 1200
N_train = 1000
N_test = N - N_train
len_sequence = 20

# Generate data
X = np.zeros((N, len_sequence))
one_indexes = choice(a=N, size=int(N / 2), replace=False)
# Half of the values in X's first column are 1, half are zero
X[one_indexes, 0] = 1  # very long term memory.
# Y is just the first column of X, so the 'rule' is that X[i][0] = Y[i], and all the inbetween 0s are meaningless
Y = X[:,0]

# Split into training and testing
X_train = X[:N_train,:]
Y_train = Y[:N_train]
X_test = X[N_train:,:]
Y_test = Y[N_train:]

In [ ]:
def prepare_sequences(x_train, y_train, window_length):
    windows = []
    windows_y = []
    for i, sequence in enumerate(x_train):
        len_seq = len(sequence)
        for window_start in range(0, len_seq - window_length + 1):
            window_end = window_start + window_length
            window = sequence[window_start:window_end]
            windows.append(window)
            windows_y.append(y_train[i])
    windows = np.array(windows)
    windows = windows.reshape((windows.shape[0], windows.shape[1], 1))
    windows_y = np.array(windows_y)
    windows_y = windows_y.reshape((windows_y.shape[0], 1))
    return windows, windows_y

In [ ]:
# Split into windows
window_length = 10
(X_train_window, Y_train_window) = prepare_sequences(X_train, Y_train, window_length)
(X_test_window, Y_test_window) = prepare_sequences(X_test, Y_test, window_length)

X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

Y_test = np.array(Y_test)
Y_test = Y_test.reshape((Y_test.shape[0], 1))

X_train = np.array(X_train)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

Y_train = np.array(Y_train)
Y_train = Y_train.reshape((Y_train.shape[0], 1))

In [ ]:
# Print shapes so we know everything is OK
print('X.shape: ', X.shape)
print('Y.shape: ', Y.shape)
print('X_train.shape: ', X_train.shape)
print('Y_train.shape: ', Y_train.shape)
print('X_test.shape: ', X_test.shape)
print('Y_test.shape: ', Y_test.shape)
print('X_train_window.shape: ', X_train_window.shape)
print('Y_train_window.shape: ', Y_train_window.shape)
print('X_test_window.shape: ', X_test_window.shape)
print('Y_test_window.shape: ', Y_test_window.shape)

In [ ]:
len(X_train)

In [ ]:
# Build first (stateless) LSTM
time_steps = window_length # np.max(X_train.shape)
batch_size = 100 # window_length
print('Building STATELESS model...')
model = Sequential()
# model.add(LSTM(10, input_shape=(time_steps, 1), return_sequences=False, stateful=False))
model.add(LSTM(10, batch_input_shape=(batch_size, time_steps, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_window, Y_train_window, batch_size=batch_size, epochs=15,
          validation_data=(X_test_window, Y_test_window), shuffle=False)

In [ ]:
score, acc = model.evaluate(X_test_window, Y_test_window, batch_size=batch_size, verbose=0)
print('Score: ', score, 'acc: ', acc)

In [ ]:
print('Build STATEFUL model...')
model = Sequential()
#model.add(LSTM(10, batch_input_shape=(1, window_length, 1), return_sequences=False, stateful=True))
model.add(LSTM(10, batch_input_shape=(1, 1, 1), return_sequences=False, stateful=True))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print('Train...')
max_len = 20
for epoch in range(15):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(X_train)):
        y_true = Y_train[i]
        for j in range(max_len):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(np.expand_dims(X_train[i][j], axis=1), axis=1),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(X_test)):
        for j in range(max_len):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1),
                                                  Y_test[i])
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(max_len):
            y_pred = model.predict_on_batch(np.expand_dims(np.expand_dims(X_test[i][j], axis=1), axis=1))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')

In [ ]:
print('Train...')
max_len = 20
num_windows = max_len-window_length+1
for epoch in range(5):
    mean_tr_acc = []
    mean_tr_loss = []
    for i in range(len(X_train)):
        y_true = Y_train[i]
        for j in range(num_windows):
            tr_loss, tr_acc = model.train_on_batch(np.expand_dims(X_train_window[i*num_windows+j], axis=0),
                                                   np.array([y_true]))
            mean_tr_acc.append(tr_acc)
            mean_tr_loss.append(tr_loss)
        model.reset_states()

    print('accuracy training = {}'.format(np.mean(mean_tr_acc)))
    print('loss training = {}'.format(np.mean(mean_tr_loss)))
    print('___________________________________')

    mean_te_acc = []
    mean_te_loss = []
    for i in range(len(X_test)):
        for j in range(num_windows):
            te_loss, te_acc = model.test_on_batch(np.expand_dims(X_test_window[i*num_windows+j], axis=0),
                                                  Y_test[i])
            mean_te_acc.append(te_acc)
            mean_te_loss.append(te_loss)
        model.reset_states()

        for j in range(num_windows):
            y_pred = model.predict_on_batch(np.expand_dims(X_test_window[i*num_windows+j], axis=0))
        model.reset_states()

    print('accuracy testing = {}'.format(np.mean(mean_te_acc)))
    print('loss testing = {}'.format(np.mean(mean_te_loss)))
    print('___________________________________')